In [1]:
#!code .

# Pricing d'option sur cryptomonnaies
#### LL

### Imports

Les imports se font de la manière suivante
- Le scraper se trouve dans le module Market
- Les modèles sont dans le module Models
- Des fonctions utiles se trouvent dans le sous-module utils du module Models, on y retrouve des fonctions d'évaluations pour les modèles

In [2]:
from Cryptocurrency_pricing.Market import deribit_data as dm
from Cryptocurrency_pricing.Models.utils import *
from Cryptocurrency_pricing.Models import BlackScholes, Merton, Heston

import numpy as np
import plotly.graph_objects as go   

### Scraping

Pour effectuer le scraping de données depuis deribit, il faut instancier la classe Scraper du module deribit_data, l'argument $\verb+currency+$ permet de choisir la cryptomonnaie sur laquelle on veut récupérer les données du marché (BTC, ETH, etc...) 

In [3]:
data = dm.Scraper(currency='BTC')

Une fois le scraper initialisé, on collecte la donnée par la méthode $\verb+collect_data+$.
- $\tt{max\_workers}$ determine le nombre de threads qui vont marcher en parallèle pour scraper la donnée (20 max pour ne pas surcharger les requêtes)
- $\tt{save\_csv = True}$ pour sauvegarder la donnée scrapée en csv.


In [4]:
raw_df = data.collect_data(max_workers = 15, save_csv = False)

Data Collected


In [5]:
raw_df.head(3)

,underlying_price,underlying_index,timestamp,stats,state,settlement_price,open_interest,min_price,max_price,mark_price,...,change_id,bids,bid_iv,best_bid_price,best_bid_amount,best_ask_price,best_ask_amount,asks,ask_iv,option_type
0,34507.37,BTC-24SEP21,1625478574014,"{'volume': 0.1, 'price_change': 0.0, 'low': 0....",open,0.27,539.1,0.2040,0.3635,0.271508,...,32733059720,"[[0.267, 0.6], [0.2665, 0.6], [0.2315, 1.0], [...",87.60,0.2670,0.6,0.2890,0.3,"[[0.289, 0.3], [0.2895, 20.2], [0.3115, 1.0], ...",99.40,P
1,36300.52,BTC-24JUN22,1625478574014,"{'volume': None, 'price_change': None, 'low': ...",open,0.41,0.0,0.3385,0.5140,0.414394,...,32733056884,"[[0.374, 12.0], [0.3405, 1.0], [0.202, 0.4], [...",79.92,0.3740,12.0,0.4570,12.0,"[[0.457, 12.0], [0.4905, 1.0]]",106.33,C
2,34222.56,BTC-9JUL21,1625478574014,"{'volume': 1.1, 'price_change': -21.978, 'low'...",open,0.10,19.7,0.0680,0.1480,0.103744,...,32733058155,"[[0.1025, 3.0], [0.1015, 1.7], [0.0975, 0.4], ...",97.46,0.1025,3.0,0.1055,3.0,"[[0.1055, 3.0], [0.106, 0.7], [0.1065, 1.0], [...",109.16,C


La donnée brute sur les options contient beaucoup d'information, elles seront traitées en amont dans les modèles.

### Initialisation des modèles

Les modèles s'initialisent de la façon suivante, avec deux paramètres, le dataframe brut scrapé, ainsi que le type d'option à garder : $$\tt{ \{C : Call, P : Put, B : Both\} }$$

In [6]:
BS = BlackScholes(raw_df.copy(), 'B')
M = Merton(raw_df.copy(), 'B')
H = Heston(raw_df.copy(), 'B')

Chaque modèle possède désormais un DataFrame $\tt{df}$ trié lors de l'initialisation des modèles par la pipeline de tri du module $\tt{utils}$, il ne contient que les informations utiles pour la suite.

In [24]:
BS.df.tail(25)

,S,K,V,_T,bids,asks,last_price,mark_price,option_type,mid_iv,mark_iv,mid,moneyness,I_VOL,IV_moneyness,BS_PRICE
308,34204.04,32000.0,40.2,0.067142,0.055593,0.130729,0.0620,0.062087,P,89.780,90.13,0.093161,1.068876,0.078430,4.361112e+05,0.082164
309,34203.79,34000.0,40.6,0.067142,0.083166,0.092200,0.0865,0.086558,P,86.245,86.30,0.087683,1.005994,0.008622,3.967054e+06,0.031739
44,34205.37,35000.0,56.6,0.067142,0.072677,0.081500,0.0782,0.078045,C,84.700,84.74,0.077089,0.977296,0.028870,1.184825e+06,0.106013
359,34203.81,36000.0,95.3,0.067142,0.059283,0.067666,0.0650,0.065356,C,83.665,83.52,0.063474,0.950106,0.059473,5.751129e+05,0.074877
367,34203.81,40000.0,258.7,0.067142,0.028087,0.036920,0.0300,0.030280,C,80.740,80.78,0.032504,0.855095,0.161114,2.122956e+05,0.034763
137,34205.32,45000.0,68.1,0.067142,0.008483,0.013979,0.0115,0.011660,C,81.750,82.06,0.011231,0.760118,0.257328,1.329248e+05,0.011751
58,34204.80,50000.0,6.7,0.067142,0.003791,0.014371,0.0060,0.005438,C,87.345,87.18,0.009081,0.684096,0.345929,9.887819e+04,0.009400
28,34206.52,60000.0,8.8,0.067142,0.001327,0.004951,0.0020,0.001940,C,100.190,100.05,0.003139,0.570109,0.477324,7.166306e+04,0.003223
166,34356.06,25000.0,7.4,0.143854,0.034718,0.039731,0.0375,0.036846,P,100.490,100.60,0.037225,1.374242,0.219122,1.567896e+05,0.034543
358,34352.39,30000.0,13.4,0.143854,0.072329,0.121997,0.0780,0.078135,P,93.240,93.35,0.097163,1.145080,0.104094,3.300141e+05,0.084125


La méthode $\tt{initialize}$ permet d'initialiser les paramètres propres à chaque modèle, depuis un vecteur $\theta$, le booléen $\tt{reset}$ permet de choisir de réinitialiser aux derniers paramètres calibrés enregistrés.

In [8]:
BS.initialize(reset = True)
M.initialize(reset = True)
H.initialize(reset = True)

### Ajouts des volatiliés implicites

La méthode append_imp_vol_to_df ajoute une colonne au DataFrame du modèle avec les volatilités implicites, calculées  dans le module $\tt{common\_all}$

In [9]:
BS.append_imp_vol_to_df()
M.append_imp_vol_to_df()
H.append_imp_vol_to_df()

### Ajouts des Prix respectifs de chaque modèles aux df 

Chaque modèle possède une méthode $\tt{Price}$ qui calcule le prix d'une option sous ce modèle.

In [29]:
S = 34205.37
K = 35000.0
T = 0.067142
v = 0.028870
Flag = 'C'
print("Black-Scholes Price : ${}".format(S*BS.Price(S=S, K=K, T=T, sigma=v, CallPutFlag=Flag)))
print("Merton Price        : ${}".format(S*M.Price(S=S, K=K, T=T, sigma=v, CallPutFlag=Flag)))
print("Heston Price        : ${}".format(S*H.Price(S=S, K=K, T=T, sigma=v, CallPutFlag=Flag)))
print("Market Price        : ${}".format(S*0.077089))

Black-Scholes Price : $3626.812075362676
Merton Price        : $3629.7543747209543
Heston Price        : $2996.9214575768083
Market Price        : $2636.8577679300006


La vectorisation de cette fonction et son application aux lignes du DataFrame permet de calculer une liste de prix qui pourront être comparés avec ceux du marché. Cela ce fait via la méthode $\tt{append\_price}$.

In [30]:
BS.append_price()
M.append_price()
H.append_price()

BS.df[['BS_PRICE','mid']].head()

,BS_PRICE,mid
213,0.001064,0.001066
27,0.001593,0.001598
24,0.002492,0.002502
388,0.005406,0.005438
227,0.012577,0.012845


In [12]:
bad_prices_indexes = H.df[H.df.mid >= 1].index


BS.df = BS.df.drop(bad_prices_indexes)
H.df = H.df.drop(bad_prices_indexes)
M.df = M.df.drop(bad_prices_indexes)

bad_prices_indexes =  BS.df[BS.df.BS_PRICE >= 1].index

BS.df = BS.df.drop(bad_prices_indexes)
H.df = H.df.drop(bad_prices_indexes)
M.df = M.df.drop(bad_prices_indexes)




In [13]:
#H.df[H.df.mid >= 0].index

### Optimisation des paramètres (pour Merton et Heston)

In [14]:
H.optimize(x0 = None, tol = 1e-4, max_iter =300, update_when_done=True )

array([0.90260584, 0.42127592, 0.13607334, 0.37337941, 0.20804514])

In [27]:
M.optimize(tol = 1e-7, max_iter=500, update_when_done=True)

array([9.17579001e-01, 3.50404358e-02, 2.89575783e-04])

In [33]:
from sklearn.metrics import mean_absolute_error

x = BS.df['K']
y = BS.df['_T']
z = BS.df['mid']

x1 = BS.df['K']
y1 = BS.df['_T']
z1 = BS.df['BS_PRICE']

x2 = M.df['K']
y2 = M.df['_T']
z2 = M.df['MERTON_PRICE']

x3 = H.df['K']
y3 = H.df['_T']
z3 = H.df['HESTON_PRICE']






fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='Real Market Price', marker=dict(opacity=0.8)),
                      go.Scatter3d(x=x1, y=y1, z=z1, mode='markers', name='BlackScholes Model Price', marker=dict(opacity=0.8)),
                      go.Scatter3d(x=x2, y=y2, z=z2, mode='markers', name='Merton Model Price', marker=dict(opacity=0.8),),
                      go.Scatter3d(x=x3, y=y3, z=z3, mode='markers', name='Heston Model Price', marker=dict(opacity=0.8))])


fig.update_scenes(xaxis_title_text='Strike', yaxis_title_text='Exp', zaxis_title_text='Price') 
fig.show()

#rms_merton = mean_squared_error(z2, z3, squared=False)



In [34]:
Eval_Metrics(BS=BS,M=M,H=H)

=============== Root Mean Squared Error ================

B&S    = 2.334 %
MERTON    = 2.128 %
HESTON    = 0.821 % 

=============== Mean Absolute Error ================

B&S    = 0.778 %
MERTON    = 0.566 %
HESTON    = 0.579 % 

=============== R2 Score ================

B&S    = 74.244 %
MERTON    = 80.441 %
HESTON    = 94.433 % 



finir git
tests unitaires
heston + sauts